<a href="https://colab.research.google.com/github/anupojuharshita/GEN_AI-project/blob/main/curve%20with%20offset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

detection


In [ ]:
import cv2
import numpy as np

def canny_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

def region_of_interest(image):
    height, width = image.shape[:2]
    polygons = np.array([
        [(100, height), (width // 2 - 80, height // 2 + 100), (width // 2 + 80, height // 2 + 100), (width - 100, height)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def fit_curve(lines, image):
    if lines is None or len(lines) < 2:
        return np.array([])

    points = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            points.append((x1, y1))
            points.append((x2, y2))

    points = np.array(points, dtype=np.int32)
    if len(points) < 5:
        return np.array([])

    y = points[:, 1]
    x = points[:, 0]
    y_normalized = y / np.max(y)

    try:
        curve = np.polyfit(y_normalized, x, 2)
    except np.linalg.LinAlgError:
        return np.array([])

    y_vals = np.linspace(image.shape[0] * 0.7, image.shape[0], num=50)
    y_vals_normalized = y_vals / np.max(y)
    x_vals = np.polyval(curve, y_vals_normalized)

    curve_points = np.array([np.column_stack((x_vals, y_vals))], dtype=np.int32)
    return curve_points

def draw_curve(image, lines, color=(0, 255, 0)):
    curve_image = np.zeros_like(image)
    curve_points = fit_curve(lines, image)
    if curve_points.size > 0:
        cv2.polylines(curve_image, curve_points, isClosed=False, color=color, thickness=5)
    return curve_image

def process_frame(frame):
    edges = canny_edge_detection(frame)
    masked_edges = region_of_interest(edges)
    lines = cv2.HoughLinesP(masked_edges, 2, np.pi/180, 80, np.array([]), minLineLength=30, maxLineGap=100)

    left_lines = []
    right_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1) if x2 - x1 != 0 else 0
            if slope < -0.3:
                left_lines.append(line)
            elif slope > 0.3:
                right_lines.append(line)

    left_curve_image = draw_curve(frame, left_lines, (255, 0, 0))
    right_curve_image = draw_curve(frame, right_lines, (0, 255, 0))

    combined = frame.copy()
    combined = cv2.addWeighted(combined, 0.8, left_curve_image, 1, 1)
    combined = cv2.addWeighted(combined, 0.8, right_curve_image, 1, 1)

    return combined

def main(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame)
        out.write(processed_frame)

    cap.release()
    out.release()
    print(f"Processed video saved as {output_path}")

if __name__ == "__main__":
    video_path = "/content/cross road.mp4"
    output_path = "/content/lane_detected_output.avi"
    main(video_path, output_path)

Processed video saved as /content/lane_detected_output.avi


area fill

In [ ]:
import cv2
import numpy as np

def canny_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

def region_of_interest(image):
    height, width = image.shape[:2]
    polygons = np.array([
        [(100, height), (width // 2 - 80, height // 2 + 100), (width // 2 + 80, height // 2 + 100), (width - 100, height)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def fit_curve(lines, image):
    if lines is None or len(lines) < 2:
        return np.array([])

    points = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            points.append((x1, y1))
            points.append((x2, y2))

    points = np.array(points, dtype=np.int32)
    if len(points) < 5:
        return np.array([])

    y = points[:, 1]
    x = points[:, 0]
    y_normalized = y / np.max(y)

    try:
        curve = np.polyfit(y_normalized, x, 2)
    except np.linalg.LinAlgError:
        return np.array([])

    y_vals = np.linspace(image.shape[0] * 0.7, image.shape[0], num=50)
    y_vals_normalized = y_vals / np.max(y)
    x_vals = np.polyval(curve, y_vals_normalized)

    curve_points = np.array([np.column_stack((x_vals, y_vals))], dtype=np.int32)
    return curve_points

def draw_filled_lane(image, left_lines, right_lines):
    lane_image = np.zeros_like(image)
    left_curve = fit_curve(left_lines, image)
    right_curve = fit_curve(right_lines, image)

    if left_curve.size > 0 and right_curve.size > 0:
        points = np.vstack((left_curve[0], np.flipud(right_curve[0])))
        cv2.fillPoly(lane_image, [points], (0, 255, 0))

    return lane_image

def process_frame(frame):
    edges = canny_edge_detection(frame)
    masked_edges = region_of_interest(edges)
    lines = cv2.HoughLinesP(masked_edges, 2, np.pi/180, 80, np.array([]), minLineLength=30, maxLineGap=100)

    left_lines = []
    right_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1) if x2 - x1 != 0 else 0
            if slope < -0.3:
                left_lines.append(line)
            elif slope > 0.3:
                right_lines.append(line)

    lane_fill_image = draw_filled_lane(frame, left_lines, right_lines)
    combined = cv2.addWeighted(frame, 0.8, lane_fill_image, 0.5, 1)

    return combined

def main(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame)
        out.write(processed_frame)

    cap.release()
    out.release()
    print(f"Processed video saved as {output_path}")

if __name__ == "__main__":
    video_path = "/content/cross road.mp4"
    output_path = "lane_detected_output.mp4"
    main(video_path, output_path)

Processed video saved as lane_detected_output.mp4


offset parameter

In [ ]:
import cv2
import numpy as np

def canny_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

def region_of_interest(image):
    height, width = image.shape[:2]
    polygons = np.array([[
        (100, height), (width // 2 - 80, height // 2 + 100),
        (width // 2 + 80, height // 2 + 100), (width - 100, height)
    ]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def fit_curve(lines, image):
    if lines is None or len(lines) < 2:
        return np.array([])

    points = np.array([(x1, y1) for line in lines for x1, y1, x2, y2 in line] +
                      [(x2, y2) for line in lines for x1, y1, x2, y2 in line], dtype=np.int32)
    if len(points) < 5:
        return np.array([])

    y = points[:, 1]
    x = points[:, 0]
    y_normalized = y / np.max(y)

    try:
        curve = np.polyfit(y_normalized, x, 2)
    except np.linalg.LinAlgError:
        return np.array([])

    y_vals = np.linspace(image.shape[0] * 0.7, image.shape[0], num=50)
    y_vals_normalized = y_vals / np.max(y)
    x_vals = np.polyval(curve, y_vals_normalized)

    return np.array([np.column_stack((x_vals, y_vals))], dtype=np.int32)

def draw_lanes_and_fill(frame, left_lines, right_lines):
    lane_overlay = np.zeros_like(frame)
    left_curve = fit_curve(left_lines, frame)
    right_curve = fit_curve(right_lines, frame)

    if left_curve.size > 0 and right_curve.size > 0:
        cv2.fillPoly(lane_overlay, [np.vstack((left_curve[0], np.flipud(right_curve[0])))], (0, 255, 0))

    if left_curve.size > 0:
        cv2.polylines(lane_overlay, left_curve, isClosed=False, color=(255, 0, 0), thickness=5)
    if right_curve.size > 0:
        cv2.polylines(lane_overlay, right_curve, isClosed=False, color=(0, 0, 255), thickness=5)

    return cv2.addWeighted(frame, 0.8, lane_overlay, 0.5, 1)

def calculate_deviation(frame, left_lines, right_lines):
    width = frame.shape[1]
    lane_center = width // 2
    if left_lines and right_lines:
        left_x = np.mean([x1 for line in left_lines for x1, _, _, _ in line])
        right_x = np.mean([x2 for line in right_lines for _, _, x2, _ in line])
        detected_center = (left_x + right_x) // 2
        deviation = ((detected_center - lane_center) / lane_center) * 100
    else:
        deviation = 0
    return deviation

def draw_overlay(frame, deviation, fps, left_detected, right_detected):
    overlay = frame.copy()
    lane_status = "Good Lane Keeping" if left_detected and right_detected else "Lane Detection Issue"
    upcoming_road = "Left Curve Ahead" if deviation > 5 else "Right Curve Ahead" if deviation < -5 else "Straight Road"

    cv2.putText(overlay, f"Deviation: {deviation:.2f}%", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(overlay, f"FPS: {fps:.2f}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(overlay, lane_status, (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if left_detected and right_detected else (0, 0, 255), 2)
    cv2.putText(overlay, f"Upcoming Road: {upcoming_road}", (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return overlay

def process_frame(frame, fps):
    edges = canny_edge_detection(frame)
    masked_edges = region_of_interest(edges)
    lines = cv2.HoughLinesP(masked_edges, 2, np.pi/180, 80, np.array([]), minLineLength=30, maxLineGap=100)

    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1) if x2 - x1 != 0 else 0
            if slope < -0.3:
                left_lines.append(line)
            elif slope > 0.3:
                right_lines.append(line)

    deviation = calculate_deviation(frame, left_lines, right_lines)
    lane_filled_frame = draw_lanes_and_fill(frame, left_lines, right_lines)
    overlay = draw_overlay(lane_filled_frame, deviation, fps, len(left_lines) > 0, len(right_lines) > 0)

    return overlay

def main(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        fps = cap.get(cv2.CAP_PROP_FPS)
        processed_frame = process_frame(frame, fps)
        out.write(processed_frame)

    cap.release()
    out.release()
    print(f"Processed video saved as {output_path}")

if __name__ == "__main__":
    video_path = "/content/cross road.mp4"
    output_path = "lane_detected_output.mp4"
    main(video_path, output_path)

Processed video saved as lane_detected_output.mp4
